- deploy via ssh
- recieve job from local via ssh
- obtain datainstances via globus
- execute job
- upload produced datainstances via globus
- return globus paths (indexes/keys?)

In [1]:
from pathlib import Path
from metasmith.agents.ssh import Agent
from metasmith.models.libraries import *

In [2]:
globus = GlobusSource.Parse("https://app.globus.org/file-manager?destination_id=64a5c402-05c4-4607-bbad-46a9c2aebd98&destination_path=%2Fhome%2Ftxyliu%2Fscratch%2F")
sockeye_globus_endpoint = globus.endpoint
sockeye_globus_endpoint

'64a5c402-05c4-4607-bbad-46a9c2aebd98'

In [3]:
agent = Agent(
    ssh_command='ssh sockeye',
    ssh_address='sockeye',
    pre = f"""\
        module load gcc/9.4.0 apptainer/1.3.1
    """,
    home='~/scratch/metasmith',
    globus_endpoint=sockeye_globus_endpoint
)

# agent = Agent(
#     ssh_command='ssh cedar',
#     ssh_address='cedar',
#     pre = f"""\
#         module load StdEnv/2023 apptainer/1.3.5
#     """,
#     home='~/metasmith',
#     workspace='~/scratch/metasmith',
# )

# agent.Deploy()

In [4]:
from local.constants import WORKSPACE_ROOT
from local.utils import LinkifyPath
CACHE = WORKSPACE_ROOT/"main/local_mock/cache"

types = DataTypeLibrary.Load(WORKSPACE_ROOT/"main/local_mock/prototypes/metagenomics.dev3.yml")
len(types)

6

In [5]:
xgdb_path = CACHE/"test.xgdb"
refdb_path = CACHE/"ref.xgdb"
_xgdb = DataInstanceLibrary()
refdb = DataInstanceLibrary()
_xgdb.Add(
    name="contigs",
    source=Source(WORKSPACE_ROOT/"scratch/test_ws/data/local/example.fna"),
    type=types["contigs"],
)
refdb.Add(
    name="reference.uniprot_sprot",
    source=Source(WORKSPACE_ROOT/"scratch/test_ws/data/local/uniprot_sprot.dmnd"),
    type=types["protein_reference_diamond"],
)
refdb.Add(
    name="container.diamond",
    source=Source(WORKSPACE_ROOT/"scratch/test_ws/data/local/diamond.oci.uri"),
    type=types["oci_image_diamond"],
)
refdb.Add(
    name="container.pprodigal",
    source=Source(WORKSPACE_ROOT/"scratch/test_ws/data/local/pprodigal.oci.uri"),
    type=types["oci_image_pprodigal"],
)
_xgdb.Save(xgdb_path)
refdb.Save(refdb_path)
LinkifyPath(xgdb_path/_xgdb._index_name)
LinkifyPath(refdb_path/refdb._index_name)
xgdb = DataInstanceLibrary.Load(xgdb_path)

./../../main/local_mock/cache/test.xgdb/info.yml
./../../main/local_mock/cache/ref.xgdb/info.yml


In [6]:
xgdb_g_path = CACHE/"testg.xgdb.yml"
refdb_g_path = CACHE/"testg.refdb.yml"

if not xgdb_g_path.exists():
    xgdb_g, mover = xgdb.PrepareMove(
        dest=GlobusSource.Parse("https://app.globus.org/file-manager?origin_id=2602486c-1e0f-47a0-be15-eec1b0ff0f96&origin_path=%2FMetasmith%2Fglobus_test3%2Fxgdb%2F").AsSource(),
    )
    mover.ExecuteTransfers()
    xgdb_g.Save(xgdb_g_path)
if not refdb_g_path.exists():
    refdb_g, mover = refdb.PrepareMove(
        dest=GlobusSource.Parse("https://app.globus.org/file-manager?origin_id=2602486c-1e0f-47a0-be15-eec1b0ff0f96&origin_path=%2FMetasmith%2Fglobus_test3%2Frefdb%2F").AsSource(),
    )
    mover.ExecuteTransfers()
    refdb_g.Save(refdb_g_path)

xgdb_g = DataInstanceLibrary.Load(xgdb_g_path)
refdb_g = DataInstanceLibrary.Load(refdb_g_path)
LinkifyPath(xgdb_g_path)
LinkifyPath(refdb_g_path)

./../../main/local_mock/cache/testg.xgdb.yml
./../../main/local_mock/cache/testg.refdb.yml


In [7]:
GlobusSource.Parse("https://app.globus.org/file-manager?destination_id=64a5c402-05c4-4607-bbad-46a9c2aebd98&destination_path=%2Fhome%2Ftxyliu%2F").endpoint

'64a5c402-05c4-4607-bbad-46a9c2aebd98'

In [8]:
trlib = TransformInstanceLibrary.Load([
    Path("./transforms/simple_1"),
    # Path("./transforms/dupe_test"),
])
for k, v in trlib:
    print(k.name, type(v))

diamond.py <class 'metasmith.models.libraries.TransformInstance'>
pprodigal.py <class 'metasmith.models.libraries.TransformInstance'>


In [9]:
from metasmith.models.workflow import WorkflowPlan

plan = WorkflowPlan.Generate(
    given=[xgdb_g, refdb_g],
    transforms=trlib,
    targets=[
        types["orf_annotations"].WithLineage([
            types["contigs"],
            # xgdb["example.fna"].type,
        ]),
    ],
)

for step in plan.steps:
    step_path = Path(step.transform._source.address)
    model = step.transform.model
    print(f"{step_path.stem}")
    for x in model.requires:
        print(f"    {x}")
    print("    v")
    for x in model.produces:
        print(f"    {x}")
    # print([f"{x.source}" for x in step.uses], [f"{x.source}" for x in step.produces], sep="->")
    # LinkifyPath(step.transform._source.address)

pprodigal
    (D:{"data":"software container"}-{"format":"OCI"}-{"provides":["pprodigal"]})
    (D:{"data":"DNA sequence"}-{"format":"FASTA"})
    v
    (D:{"data":"Amino acid sequence"}-{"format":"FASTA"})
diamond
    (D:{"data":"software container"}-{"format":"OCI"}-{"provides":["diamond"]})
    (D:{"data":"Amino acid sequence"}-{"format":"FASTA"})
    (D:{"data":"database reference"}-{"format":".dmnd"})
    v
    (D:{"data":"Protein features"}-{"format":"CSV"})


In [10]:
plan._key

'VLr5Nw'

In [11]:
from metasmith.coms.ipc import LiveShell
from metasmith.logging import Log
from tempfile import TemporaryDirectory

from metasmith.models.workflow import WorkflowTask

with open(WORKSPACE_ROOT/"secrets/slurm_account") as f:
    slurm_account = f.read().strip()

plan = plan
local_temp = Path("./cache/mock_temp/")
# with LiveShell() as shell, TemporaryDirectory(prefix="msm.") as local_temp:
with LiveShell() as sh_local, LiveShell() as sh_remote:
    local_temp = Path(local_temp)
    sh_local.RegisterOnOut(lambda x: Log.Info(f"{x}"))
    sh_local.RegisterOnErr(lambda x: Log.Error(f"{x}"))
    sh_remote.RegisterOnOut(lambda x: print(f"R| {x}"))
    sh_remote.RegisterOnErr(lambda x: print(f"R| {x}"))
    remote_cache = Path(f"/tmp/metasmith.{plan._key}")

    task_path = local_temp/f"plan.json"
    task = WorkflowTask(plan=plan, agent=agent, config=dict(
        nextflow=dict(
            slurm_account=slurm_account
        ),
    ))
    task.Save(task_path)

    tr_path = local_temp/f"transforms.lib"
    trlib.Gather(tr_path, overwrite=True)

    Log.Info(f"establishing ssh via [{agent.ssh_command}]")
    res = sh_remote.Exec(agent.ssh_command)
    errs = [e for e in res.err if e not in {"Pseudo-terminal will not be allocated because stdin is not a terminal."}]
    assert len(errs) == 0, errs
    Log.Info(f"runnning setup commands:")
    for l in agent.pre.split("\n"):
        Log.Info(f"    {l}")
    sh_remote.Exec(agent.pre)
    Log.Info(f"making destination cache at [{remote_cache}]")
    sh_remote.Exec(f"[ -e {remote_cache} ] && rm -rf {remote_cache}")
    sh_remote.Exec(f"mkdir -p {remote_cache} && cd {agent.home}")
    Log.Info(f"starting relay service")
    sh_remote.Exec(f"./relay/msm_relay start")

    task_dest = f"{agent.ssh_address}:{remote_cache/task_path.name}"
    Log.Info(f"sending task [{task_path}] to [{task_dest}]")
    sh_local.Exec(f"rsync -avcp {task_path} {task_dest}")
    tr_dest = f"{agent.ssh_address}:{remote_cache/tr_path.name}"
    Log.Info(f"sending transforms [{tr_path}] to [{tr_dest}]")
    sh_local.Exec(f"rsync -avcp --exclude=__pycache__ {tr_path}/ {tr_dest}") # the trailing slash is important

    Log.Info(f"calling stage")
    sh_remote.Exec(f'./msm api stage_workflow -a plan_dir={remote_cache} -a force=True') # this is ok because /tmp is mounted
    # Log.Info(f"calling execute")
    # sh_remote.Exec(f'./msm api execute_workflow -a plan_dir=/ws/runs/{plan._key}')

    sh_remote.Exec(f"rm -rf {remote_cache}")
    # sh_remote.Exec(f"{agent.home}/relay/msm_relay stop")
    sh_remote.Exec("exit")

2025-02-21_17-52-15  | starting [3] transfers
2025-02-21_17-52-15  | executing [3] local transfers
2025-02-21_17-52-16  | attempted [3] transfers
2025-02-21_17-52-16  | establishing ssh via [ssh sockeye]
R| Pseudo-terminal will not be allocated because stdin is not a terminal.
2025-02-21_17-52-17  | runnning setup commands:
2025-02-21_17-52-17  |     module load gcc/9.4.0 apptainer/1.3.1
2025-02-21_17-52-18  | making destination cache at [/tmp/metasmith.VLr5Nw]
2025-02-21_17-52-18  | starting relay service
R| 2025-02-21_17-52-18 E| relay server already running in [relay/connections]
2025-02-21_17-52-19  | sending task [cache/mock_temp/plan.json] to [sockeye:/tmp/metasmith.VLr5Nw/plan.json]
2025-02-21_17-52-20  | sending incremental file list
2025-02-21_17-52-20  | plan.json
2025-02-21_17-52-20  | sent 7,749 bytes  received 35 bytes  5,189.33 bytes/sec
2025-02-21_17-52-20  | total size is 7,625  speedup is 0.98
2025-02-21_17-52-20  | sending transforms [cache/mock_temp/transforms.lib] t

In [17]:
x1 = Source("asdf")
x2 = Source("1234")

k1 = (x1, x2)
x = [k1]
x.remove((x1, x2))

In [13]:
for s in task.plan.steps:
    print(s.transform_key)

DL05P
0RiTl


In [14]:
# error here
# path incorrect
# loading type library [/ws/runs/WSFa2f/_metasmith/transforms.lib/_type_libraries/metagenomics.dev3.yml]

In [15]:
GlobusSource.Parse(task.plan.given[0].source.address)

globus://2602486c-1e0f-47a0-be15-eec1b0ff0f96:/Metasmith/globus_test3/xgdb/example.msm_sbq72Lgs35k6.fna

In [16]:
"https://app.globus.org/file-manager?origin_id=2602486c-1e0f-47a0-be15-eec1b0ff0f96&origin_path=%2FMetasmith%2Fglobus_test1%2F"

'https://app.globus.org/file-manager?origin_id=2602486c-1e0f-47a0-be15-eec1b0ff0f96&origin_path=%2FMetasmith%2Fglobus_test1%2F'